In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
data = pd.read_pickle('Database/attributes/train_val_test_for_moiverating.pkl')
CF_df = pd.read_csv('Database/attributes/expanded_recommendations.csv')

In [3]:
train = pd.concat([data['x_train'], data['y_train']], axis=1)
valid = pd.concat([data['x_val'], data['y_val']], axis=1)
test = pd.concat([data['x_test'], data['y_test']], axis=1)

In [4]:
set_df1 = CF_df[['id', 'item']]
set_df2 = CF_df[['id', 'recommended_item']]
set_df1 = set_df1.drop_duplicates()
set_df1.columns = ['userId', 'movieId']
set_df2 = set_df2.drop_duplicates()
set_df2.columns = ['userId', 'movieId']

In [5]:
total_df = pd.concat([set_df1, set_df2], axis=0)
total_df = total_df.drop_duplicates().reset_index(drop=True)

In [6]:
mask1 = train.set_index(['userId', 'movieId']).index.isin(total_df.set_index(['userId', 'movieId']).index)
mask2 = valid.set_index(['userId', 'movieId']).index.isin(total_df.set_index(['userId', 'movieId']).index)
mask3 = test.set_index(['userId', 'movieId']).index.isin(total_df.set_index(['userId', 'movieId']).index)
real_test = pd.concat([train[mask1], valid[mask2], test[mask3]], axis=0)
real_test = real_test.reset_index(drop=True)

In [7]:
real_train = pd.DataFrame()
train_dict = {}
for i in np.arange(0.5, 5.5, 0.5):
    train_dict[i] = train[~mask1 & (train['rating'] == i)].sample(n=300000, random_state=42)
    real_train = pd.concat([real_train, train_dict[i]], axis=0)
real_train = real_train.reset_index(drop=True)

In [8]:
# real_test['rating'] = real_test['rating'].apply(lambda x: int(x * 2) - 1)
# real_train['rating'] = real_train['rating'].apply(lambda x: int(x * 2) - 1)

In [9]:
from sklearn.model_selection import train_test_split

x = real_train.drop(columns='rating')
y = real_train['rating']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

x_test = real_test.drop(columns='rating')
y_test = real_test['rating']

In [10]:
# 데이터 저장 경로
train_val_test_split = {
    "x_train": x_train,
    "y_train": y_train,
    "x_val": x_val,
    "y_val": y_val,
    "x_test": x_test,
    "y_test": y_test
}

# pickle 파일로 저장
file_path = "Database/train_val_test2.pkl"
with open(file_path, "wb") as f:
    pickle.dump(train_val_test_split, f)

In [11]:
list(real_train.columns)

['userId',
 'movieId',
 'genres',
 'release_date',
 'popularity',
 'runtime',
 'revenue',
 'rating']

In [12]:
list(real_train.nunique().values)

[229812, 29238, 20, 13077, 28922, 322, 6563, 10]